In [1]:
import sympy as sp

# Pretty printing in notebooks
sp.init_printing(use_unicode=True)

# ## Symbols and indexed control points

# %%
t = sp.symbols('t', real=True)
k = sp.symbols('k', integer=True)

# Indexed bases for x/y components
Qx = sp.IndexedBase('Qx')
Qy = sp.IndexedBase('Qy')
P1x = sp.IndexedBase('P1x')
P1y = sp.IndexedBase('P1y')
P2x = sp.IndexedBase('P2x')
P2y = sp.IndexedBase('P2y')

# %% [markdown]
# ## Build the 2D control points for segment k

# %%
P0 = sp.Matrix([Qx[k],    Qy[k]])          # Q_k
P1 = sp.Matrix([P1x[k],   P1y[k]])         # P1_k
P2 = sp.Matrix([P2x[k],   P2y[k]])         # P2_k
P3 = sp.Matrix([Qx[k+1],  Qy[k+1]])        # Q_{k+1}

P0, P1, P2, P3


⎛⎡Qx[k]⎤  ⎡P1x[k]⎤  ⎡P2x[k]⎤  ⎡Qx[k + 1]⎤⎞
⎜⎢     ⎥, ⎢      ⎥, ⎢      ⎥, ⎢         ⎥⎟
⎝⎣Qy[k]⎦  ⎣P1y[k]⎦  ⎣P2y[k]⎦  ⎣Qy[k + 1]⎦⎠

In [50]:
# Print Name = Expression
def spprint(name, expression):
    sym = sp.Symbol(name)
    display(sp.Eq(sym, expression, evaluate=False))

In [65]:
# ## Define cubic Bézier segment \(B_k(t)\) as a vector function

# %%
B = (1 - t)**3 * P0 \
  + 3*(1 - t)**2 * t * P1 \
  + 3*(1 - t) * t**2 * P2 \
  + t**3 * P3

spprint('B_(t)', B)

       ⎡ 3              2                               2                 3    ↪
       ⎢t ⋅Qx[k + 1] + t ⋅(3 - 3⋅t)⋅P2x[k] + 3⋅t⋅(1 - t) ⋅P1x[k] + (1 - t) ⋅Qx ↪
B₍ₜ₎ = ⎢                                                                       ↪
       ⎢ 3              2                               2                 3    ↪
       ⎣t ⋅Qy[k + 1] + t ⋅(3 - 3⋅t)⋅P2y[k] + 3⋅t⋅(1 - t) ⋅P1y[k] + (1 - t) ⋅Qy ↪

↪    ⎤
↪ [k]⎥
↪    ⎥
↪    ⎥
↪ [k]⎦

In [64]:
# ## Compute second derivative \(B_k''(t)\)

# %%
Bpp = sp.diff(B, t, 2)
spprint('B\'\'_(t)', Bpp)

         ⎡6⋅(t⋅P1x[k] - 2⋅t⋅P2x[k] + t⋅Qx[k + 1] + 2⋅(t - 1)⋅P1x[k] - (t - 1)⋅ ↪
B''₍ₜ₎ = ⎢                                                                     ↪
         ⎣6⋅(t⋅P1y[k] - 2⋅t⋅P2y[k] + t⋅Qy[k + 1] + 2⋅(t - 1)⋅P1y[k] - (t - 1)⋅ ↪

↪ P2x[k] - (t - 1)⋅Qx[k])⎤
↪                        ⎥
↪ P2y[k] - (t - 1)⋅Qy[k])⎦

In [28]:
Bpp.subs(t,0)

⎡-12⋅P1x[k] + 6⋅P2x[k] + 6⋅Qx[k]⎤
⎢                               ⎥
⎣-12⋅P1y[k] + 6⋅P2y[k] + 6⋅Qy[k]⎦

In [41]:
c1_constraint = {P2[0]:2*Qx[k+1]-P1x[k+1], \
                 P2[1]:2*Qy[k+1]-P1y[k+1]}
spprint('c¹_constrarint', c1_constraint)

In [5]:
Bpp = Bpp.subs(c1_constraint)
Bpp

⎡-12⋅t⋅(-P1x[k + 1] + 2⋅Qx[k + 1]) + 6⋅t⋅P1x[k] + 6⋅t⋅Qx[k + 1] - 6⋅(t - 1)⋅(- ↪
⎢                                                                              ↪
⎣-12⋅t⋅(-P1y[k + 1] + 2⋅Qy[k + 1]) + 6⋅t⋅P1y[k] + 6⋅t⋅Qy[k + 1] - 6⋅(t - 1)⋅(- ↪

↪ P1x[k + 1] + 2⋅Qx[k + 1]) + 12⋅(t - 1)⋅P1x[k] - 6⋅(t - 1)⋅Qx[k]⎤
↪                                                                ⎥
↪ P1y[k + 1] + 2⋅Qy[k + 1]) + 12⋅(t - 1)⋅P1y[k] - 6⋅(t - 1)⋅Qy[k]⎦

In [53]:
# ## Squared Euclidean norm \(\|B_k''(t)\|^2\)
# 
# Use dot product (no square roots, no Abs).

# %%
sqnorm = sp.simplify(Bpp.dot(Bpp))
sqnorm

In [54]:
# ## Integrate over \([0,1]\) to get bending energy \(I_k\)

# %%
Ik = sp.simplify(sp.integrate(sqnorm, (t, 0, 1)))
spprint('I_k', Ik)

In [55]:
def bending_energy_segment_2d(expr, k_value=None, subs=None):
    """
    Return symbolic I_k.
    - k_value: optionally substitute a concrete integer for k
    - subs: dict of further substitutions (numeric or symbolic)
    """
    #expr = Ik
    if k_value is not None:
        expr = expr.subs(k, k_value)
    if subs is not None:
        expr = expr.subs(subs)
    return sp.simplify(expr)

In [56]:
L = bending_energy_segment_2d(Ik)
spprint('L', L)

### First Segment of the Spline

In [57]:
D1 = L + L.subs({k:k+1})
spprint('D_1', D1)

In [58]:
grad1 = sp.Matrix([D1.diff(P1x[k]), D1.diff(P1y[k])])
spprint('grad_1', grad1)

        ⎡72⋅P1x[k] - 36⋅P2x[k] - 36⋅Qx[k]⎤
grad₁ = ⎢                                ⎥
        ⎣72⋅P1y[k] - 36⋅P2y[k] - 36⋅Qy[k]⎦

In [12]:
sp.Eq(grad1[0]/36, 0)

In [13]:
sp.Eq(grad1[0]/36, 0).subs(k, 0)

In [66]:
D1 = L.subs(k,0) + L.subs({k:1})
spprint('D_1', D1)

In [67]:
grad1 = sp.Matrix([D1.diff(P1x[0]), D1.diff(P1y[0])])
spprint('grad_1', grad1)

        ⎡72⋅P1x[0] - 36⋅P2x[0] - 36⋅Qx[0]⎤
grad₁ = ⎢                                ⎥
        ⎣72⋅P1y[0] - 36⋅P2y[0] - 36⋅Qy[0]⎦

### Inner Segments of the Spline

In [62]:
D2 = L + L.subs({k:k-1})
spprint('D_2', D2)

In [15]:
grad2 = sp.Matrix([D2.diff(P1x[k]), D2.diff(P1y[k])])
grad2

⎡36⋅P1x[k + 1] + 36⋅P1x[k - 1] + 144⋅P1x[k] - 72⋅Qx[k + 1] - 144⋅Qx[k]⎤
⎢                                                                     ⎥
⎣36⋅P1y[k + 1] + 36⋅P1y[k - 1] + 144⋅P1y[k] - 72⋅Qy[k + 1] - 144⋅Qy[k]⎦

In [16]:
sp.Eq(grad2[0]/36, 0)

### Last Segment of the Spline

In [17]:
n = sp.symbols('n', integer=True)

In [18]:
c2_constraint = {P2[0]:(Qx[k+1]+P1x[k])/2, \
                 P2[1]:(Qy[k+1]+P1y[k])/2}
c2_constraint

In [19]:
Bpp = sp.diff(B, t, 2)
Bpp

⎡6⋅(t⋅P1x[k] - 2⋅t⋅P2x[k] + t⋅Qx[k + 1] + 2⋅(t - 1)⋅P1x[k] - (t - 1)⋅P2x[k] -  ↪
⎢                                                                              ↪
⎣6⋅(t⋅P1y[k] - 2⋅t⋅P2y[k] + t⋅Qy[k + 1] + 2⋅(t - 1)⋅P1y[k] - (t - 1)⋅P2y[k] -  ↪

↪ (t - 1)⋅Qx[k])⎤
↪               ⎥
↪ (t - 1)⋅Qy[k])⎦

In [20]:
Bpp = Bpp.subs(c2_constraint)
Bpp

⎡       ⎛P1x[k]   Qx[k + 1]⎞                                          ⎛P1x[k]  ↪
⎢- 12⋅t⋅⎜────── + ─────────⎟ + 6⋅t⋅P1x[k] + 6⋅t⋅Qx[k + 1] - 6⋅(t - 1)⋅⎜──────  ↪
⎢       ⎝  2          2    ⎠                                          ⎝  2     ↪
⎢                                                                              ↪
⎢       ⎛P1y[k]   Qy[k + 1]⎞                                          ⎛P1y[k]  ↪
⎢- 12⋅t⋅⎜────── + ─────────⎟ + 6⋅t⋅P1y[k] + 6⋅t⋅Qy[k + 1] - 6⋅(t - 1)⋅⎜──────  ↪
⎣       ⎝  2          2    ⎠                                          ⎝  2     ↪

↪   Qx[k + 1]⎞                                      ⎤
↪ + ─────────⎟ + 12⋅(t - 1)⋅P1x[k] - 6⋅(t - 1)⋅Qx[k]⎥
↪       2    ⎠                                      ⎥
↪                                                   ⎥
↪   Qy[k + 1]⎞                                      ⎥
↪ + ─────────⎟ + 12⋅(t - 1)⋅P1y[k] - 6⋅(t - 1)⋅Qy[k]⎥
↪       2    ⎠                                      ⎦

In [21]:
sqnorm = sp.simplify(Bpp.dot(Bpp))
sqnorm

In [22]:
Ik_last = sp.simplify(sp.integrate(sqnorm, (t, 0, 1)))
Ik_last

In [23]:
D3 = bending_energy_segment_2d(Ik).subs(k, n-2) + bending_energy_segment_2d(Ik_last).subs(k, n-1)
D3

In [25]:
grad3 = sp.Matrix([D3.diff(P1x[n-1]), D3.diff(P1y[n-1])])
grad3

⎡126⋅P1x[n - 1] + 36⋅P1x[n - 2] - 144⋅Qx[n - 1] - 18⋅Qx[n]⎤
⎢                                                         ⎥
⎣126⋅P1y[n - 1] + 36⋅P1y[n - 2] - 144⋅Qy[n - 1] - 18⋅Qy[n]⎦

In [26]:
grad3/18

⎡7⋅P1x[n - 1] + 2⋅P1x[n - 2] - 8⋅Qx[n - 1] - Qx[n]⎤
⎢                                                 ⎥
⎣7⋅P1y[n - 1] + 2⋅P1y[n - 2] - 8⋅Qy[n - 1] - Qy[n]⎦